In [1]:
"""
This example builds a simple model without training.
It is converted into ONNX. Predictions are compared to
the predictions from tensorflow to check there is no
discrepencies. Inferencing time is also compared between
*onnxruntime*, *tensorflow* and *tensorflow.lite*.
"""
from onnxruntime import InferenceSession
import os
import subprocess
import timeit
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Input

########################################
# Creates the model.
model = keras.Sequential()
model.add(Input((4, 4)))
model.add(layers.SimpleRNN(8))
model.add(layers.Dense(2))
print(model.summary())
input_names = [n.name for n in model.inputs]
output_names = [n.name for n in model.outputs]
print('inputs:', input_names)
print('outputs:', output_names)

########################################
# Training
# ....
# Skipped.

########################################
# Testing the model.
input = np.random.randn(2, 4, 4).astype(np.float32)
expected = model.predict(input)
print(expected)

########################################
# Saves the model.
if not os.path.exists("dann_rnn"):
    os.mkdir("dann_rnn")
tf.keras.models.save_model(model, "dann_rnn")

########################################
# Run the command line.
proc = subprocess.run('python -m tf2onnx.convert --saved-model dann_rnn '
                      '--output dann_rnn.onnx --opset 12'.split(),
                      capture_output=True)
print(proc.returncode)
print(proc.stdout.decode('ascii'))
print(proc.stderr.decode('ascii'))

########################################
# Runs onnxruntime.
session = InferenceSession("dann_rnn.onnx")
got = session.run(None, {'input_1': input})
print(got[0])

########################################
# Measures the differences.
print(np.abs(got[0] - expected).max())

########################################
# Measures processing time.
print('tf:', timeit.timeit('model.predict(input)',
                           number=100, globals=globals()))
print('ort:', timeit.timeit("session.run(None, {'input_1': input})",
                            number=100, globals=globals()))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 8)                 104       
_________________________________________________________________
dense (Dense)                (None, 2)                 18        
Total params: 122
Trainable params: 122
Non-trainable params: 0
_________________________________________________________________
None
inputs: ['input_1']
outputs: ['dense/BiasAdd:0']
[[ 0.11663369 -0.3552611 ]
 [ 0.837031   -1.3583459 ]]
INFO:tensorflow:Assets written to: dann_rnn\assets
0

2021-10-26 13:34:44.824640: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2021-10-26 13:34:44.824659: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-10-26 13:34:46.

tf: 2.518509700000001
ort: 0.009998599999999414
